In [1]:
import os
import json
import pandas as pd
from tqdm import tqdm

from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate
from llama_index.core.evaluation import CorrectnessEvaluator

from utils import json_load

## data prepare

In [2]:
# load ground-truth question and answer pair
dataset_dir = os.path.join('data', 'source')
dataset_file_path = os.path.join(dataset_dir, 'lee_course0_rag_dataset.csv')
df = pd.read_csv(dataset_file_path)
print(df.shape)
df.head(3)

(8, 5)


,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,這門課希望學生從使用者變成什麼？,['各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工...,理解者，甚至是創造者。,ai (gpt-5-mini),ai (gpt-5-mini)
1,剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用哪個工具？,['第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握A...,答案：Google 的 VO3,ai (gpt-5-mini),ai (gpt-5-mini)
2,哪個軟體被用來合成講師的聲音？,['也是VO3自己生成的\n他產生的影片是帶有聲音的\n那其他中文講課的聲音也通通都是合成的...,答案：Eleven Labs。,ai (gpt-5-mini),ai (gpt-5-mini)


In [3]:
# load llm response
response_dir = os.path.join('data', 'source')
#response_file_path = os.path.join(response_dir, 'baseline_rag_result_R3.json')
response_file_path = os.path.join(response_dir, 'baseline_rag_result.json')
response = json_load(response_file_path)
response_df = pd.DataFrame(response)
df = pd.merge(df, response_df, how='left', on='query')
df.head(3)

load data from: data/source/baseline_rag_result.json


,query,reference_contexts,reference_answer,reference_answer_by,query_by,response,retrieved_nodes,reranked_nodes
0,這門課希望學生從使用者變成什麼？,['各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工...,理解者，甚至是創造者。,ai (gpt-5-mini),ai (gpt-5-mini),這門課希望學生從使用者變成理解者，甚至是創造者。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'eba16e20-d000-4b3e-b05a-b1589f6b9fe9'...
1,剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用哪個工具？,['第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握A...,答案：Google 的 VO3,ai (gpt-5-mini),ai (gpt-5-mini),剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用Google的VO3。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'c9b0c2ac-d7b7-4183-9bcd-6c3d57686b61'...
2,哪個軟體被用來合成講師的聲音？,['也是VO3自己生成的\n他產生的影片是帶有聲音的\n那其他中文講課的聲音也通通都是合成的...,答案：Eleven Labs。,ai (gpt-5-mini),ai (gpt-5-mini),Eleven Labs這個軟體被用來合成講師的聲音。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'c9b0c2ac-d7b7-4183-9bcd-6c3d57686b61'...


In [4]:
pipeline_name = os.path.splitext(os.path.basename(response_file_path))[0]
print(pipeline_name)
save_dir = os.path.join('data', 'temp')
os.makedirs(save_dir, exist_ok=True)
save_file_path = os.path.join(save_dir, f"{pipeline_name}_correctness.csv")

baseline_rag_result


In [5]:
llm = OpenAI(model="gpt-5-mini", temperature=0, is_streaming=False)
correct_evaluator = CorrectnessEvaluator(llm=llm, score_threshold=4.0)

In [6]:
correct_score = []
correct_feedback = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    query = row.query
    reference = row.reference_answer
    response = row.response
    result = correct_evaluator.evaluate(
        query=query,
        response=response,
        reference=reference,
    )
    correct_score.append(result.score)
    correct_feedback.append(result.feedback)

df['correct_score'] = correct_score
df['correct_feedback'] = correct_feedback
df.to_csv(save_file_path, index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:47<00:00,  5.91s/it]


In [7]:
df

,query,reference_contexts,reference_answer,reference_answer_by,query_by,response,retrieved_nodes,reranked_nodes,correct_score,correct_feedback
0,這門課希望學生從使用者變成什麼？,['各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工...,理解者，甚至是創造者。,ai (gpt-5-mini),ai (gpt-5-mini),這門課希望學生從使用者變成理解者，甚至是創造者。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'eba16e20-d000-4b3e-b05a-b1589f6b9fe9'...,5.0,Generated answer matches the reference exactly...
1,剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用哪個工具？,['第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握A...,答案：Google 的 VO3,ai (gpt-5-mini),ai (gpt-5-mini),剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用Google的VO3。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'c9b0c2ac-d7b7-4183-9bcd-6c3d57686b61'...,5.0,生成的回答與使用者問題直接相關，內容正確無誤，與參考答案一致。
2,哪個軟體被用來合成講師的聲音？,['也是VO3自己生成的\n他產生的影片是帶有聲音的\n那其他中文講課的聲音也通通都是合成的...,答案：Eleven Labs。,ai (gpt-5-mini),ai (gpt-5-mini),Eleven Labs這個軟體被用來合成講師的聲音。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'c9b0c2ac-d7b7-4183-9bcd-6c3d57686b61'...,5.0,回答與參考答案完全一致，正確且簡潔明確。
3,這門課的上課時間是週五下午幾點幾分開始？,['這門課的所有的資訊\n包括上課錄影\n課程投影片\n還有作業的講解\n你其實都可以在課程...,週五下午2點20分開始。,ai (gpt-5-mini),ai (gpt-5-mini),這門課的上課時間是週五下午2點20分開始。,[{'id': 'f1b8dce2-5b45-4ed8-b44d-0bf6e2306877'...,[{'id': 'f0e22525-58ae-443d-8c9f-024c8a93b944'...,5.0,回答直接且正確地回應了使用者的問題，與參考答案一致（週五下午2點20分開始）。
4,前兩個作業的截止日期延後到哪一天繳交？,['助教\n只要你寄信給助教說你要旁聽\n我們都可以把你加入NTU Cool\n那怎麼找到N...,10月17號。,ai (gpt-5-mini),ai (gpt-5-mini),前兩個作業的截止日期延後到10月17號繳交。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'f1b8dce2-5b45-4ed8-b44d-0bf6e2306877'...,5.0,回答直接且正確，明確指出延期後的截止日為10月17號，與參考答案一致。
5,助教預計在幾月幾號把成績送出？,['送出成績的時間\n一定是在1月9號之後\n那我們目前預計1月12號把成績送出\n那如果你...,1月12號,ai (gpt-5-mini),ai (gpt-5-mini),助教預計在1月12號把成績送出。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'f1b8dce2-5b45-4ed8-b44d-0bf6e2306877'...,5.0,答案與參考答案一致，直接且正確地回覆了助教預計送出成績的日期（1月12號）。
6,課程中目前保留需要訓練多少時間的作業？,['有需要訓練模型的作業\n就是特別花時間的作業\n訓練模型就是一件非常花時間的事情\n有一...,課程中目前保留需要訓練三到四個小時的作業。,ai (gpt-5-mini),ai (gpt-5-mini),課程中保留需要訓練模型的作業，這些作業通常需要特別花時間，可能長達三到四個小時才能完成。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': 'cb9fc3f6-be14-46c2-a608-03a6bfd9d99d'...,5.0,回覆與參考答案內容一致，明確指出作業需約三到四個小時的訓練時間，且語意通順、相關且正確。
7,如果你對生成式AI一無所知，建議先看哪一門課程？,['幫助你在未來面對更大的挑戰\n幫助你在未來面對挑戰的時候\n做好心理準備\n另外呢\n鼓...,生成式AI導論2024 的課程。,ai (gpt-5-mini),ai (gpt-5-mini),建議先看生成式AI導論2024的課程。,[{'id': '18773912-c8b5-4fb4-8d4b-cb4e38758012'...,[{'id': '8c34bf19-2fb0-4755-8b8b-86746a99481a'...,5.0,The generated answer exactly matches the refer...
